In [1]:
### Dependencies
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.preprocessing import StandardScaler

In [2]:
### Data Loading

def load_csv(file_name: str) -> pd.DataFrame:
    """
    Load a CSV file from the data directory.
    """
    data_path = Path.cwd().parent / "data" / file_name 
    if not data_path.exists():
        raise FileNotFoundError(f"{data_path} not found.")

    df = pd.read_csv(data_path, parse_dates=['Date'])
    
    # Parse 'Date' column explicitly
    # MM/DD/YY format
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y', errors='coerce')
    
    # Correct future years (pandas may interpret '30' as 2030)
    # Assume dates from 1928–2020
    df.loc[df['Date'] > pd.Timestamp.today(), 'Date'] -= pd.offsets.DateOffset(years=100)
    
    # Sort by date
    df = df.sort_values('Date').reset_index(drop=True)
    return df 



def add_features(df: pd.DataFrame, rolling_window: int = 10):
    """
    Add features for ML: returns, rolling volatility, price/volume features, lags.
    Handles NaN and infinite values, ready for scaling.
    """
    df = df.copy()
    
    # Returns and volatility
    df['Return'] = df['Adj Close'].pct_change()
    df['Volatility'] = df['Return'].rolling(rolling_window).std()

    # Price-based features
    df['High_Low_pct'] = (df['High'] - df['Low']) / df['Close']
    df['Close_Open_pct'] = (df['Close'] - df['Open']) / df['Open']
    df['MA5'] = df['Close'].rolling(5).mean()
    df['MA10'] = df['Close'].rolling(10).mean()

    # Volume-based features
    df['Volume_pct_change'] = df['Volume'].pct_change()
    df['Volume_MA5'] = df['Volume'].rolling(5).mean()

    # Lag features
    df['Return_lag1'] = df['Return'].shift(1)
    df['Return_lag2'] = df['Return'].shift(2)
    df['Volatility_lag1'] = df['Volatility'].shift(1)

    # Target: 10 days-ahead volatility
    df['Volatility_future'] = df['Volatility'].shift(-10)

    # Replace inf with NaN
    df.replace([np.inf, -np.inf], np.nan, inplace=True)

    # Fill volume-related NaNs (common in early data)
    df['Volume_pct_change'] = df['Volume_pct_change'].fillna(0)
    df['Volume_MA5'] = df['Volume_MA5'].ffill()

    # Drop only rows where the key features are missing
    df = df[df['Volatility'].notna() & df['Volatility_future'].notna()]
    df.reset_index(drop=True, inplace=True)

    # Features to scale
    features = [
        'Return', 'High_Low_pct', 'Close_Open_pct', 'MA5', 'MA10',
        'Volume_pct_change', 'Volume_MA5', 'Return_lag1', 'Return_lag2', 'Volatility_lag1'
    ]

    # Scaling
    scaler = StandardScaler()
    df[features] = scaler.fit_transform(df[features].values)

    target_col = 'Volatility_future'
    return df, features, target_col


# Load raw data
df = load_csv("SPX.csv")

# Preprocess
df_prepared, feature_cols, target_col = add_features(df)

X = df_prepared[feature_cols]
y = df_prepared[target_col]

# Drop any row where X or y has NaN
mask = (~X.isna().any(axis=1)) & (~y.isna())
X = X[mask]
y = y[mask]

print(X.head(), y.head())


# Split data into train (80%) and test (20%) by time
split_idx = int(len(df_prepared) * 0.8)

X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")

/tmp/ipykernel_85901/3369933274.py:11: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(data_path, parse_dates=['Date'])


     Return  High_Low_pct  Close_Open_pct       MA5      MA10  \
1  0.023311      -0.92304       -0.023226 -0.647124 -0.647122   
2 -0.217569      -0.92304       -0.023226 -0.647148 -0.647185   
3  0.554534      -0.92304       -0.023226 -0.647173 -0.647209   
4  0.454640      -0.92304       -0.023226 -0.647201 -0.647234   
5  0.737950      -0.92304       -0.023226 -0.647104 -0.647214   

   Volume_pct_change  Volume_MA5  Return_lag1  Return_lag2  Volatility_lag1  
1          -0.060194   -0.522899    -1.399673     0.499832        -0.166255  
2          -0.060194   -0.522899     0.023281    -1.399676        -0.217933  
3          -0.060194   -0.522899    -0.217598     0.023273        -0.217969  
4          -0.060194   -0.522899     0.554501    -0.217605        -0.203854  
5          -0.060194   -0.522899     0.454607     0.554491        -0.210628   1    0.007275
2    0.007273
3    0.007181
4    0.008303
5    0.007715
Name: Volatility_future, dtype: float64
Train size: 18642, Test size: 4

In [3]:
# Train Linear Regression Model (Richard)

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Initialize and fit the model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print results
print(f"Mean Squared Error: {mse:.6f}")
print(f"Accuracy (R2): {r2 * 100:.2f}%")

# First predictions
print("-------------------")
print("Sample predictions:")
print(y_pred[:5])

Mean Squared Error: 0.000036
Accuracy (R2): 40.08%
-------------------
Sample predictions:
[0.01309724 0.01442836 0.01458687 0.01409432 0.01318505]
